In [1]:
import numpy as np
import pandas as pd

pitch = pd.read_csv("pitching_6agg.csv")
pitch.head()

,player_id,start_year,g,w,l,gs,cg,sho,sv,ipouts,...,ibb,wp,hbp,bk,bfp,gf,r,sh,sf,tbf
0,gregelu01,2009,435,22,27,0,0,0,19,1258,...,22,16,14,0,1702,76,145,21,5,1540
1,mintogr01,1975,141,10,15,6,0,0,23,730,...,18,22,5,6,1020,64,107,NaN,NaN,919
2,towerjo01,2001,112,40,45,97,4,2,1,1857,...,10,5,28,3,2664,6,365,10,22,2504
3,hillesh01,1987,163,21,32,51,1,1,10,1364,...,15,20,9,1,1977,53,236,NaN,NaN,1763
4,contrjo01,2003,163,65,50,152,5,3,0,2856,...,9,64,50,2,4121,4,531,24,29,3677


In [2]:
rest_data = pd.read_csv("pitching_rest_agg.csv")
rest_data.head()

,player_id,end_year,g,w,l,gs,cg,sho,sv,ipouts,...,ibb,wp,hbp,bk,bfp,gf,r,sh,sf,tbf
0,gregelu01,2015,64,7,3,0,0,0,31,183,...,2,1,2,0,239,53,24,2,0,225
1,mintogr01,1990,569,49,50,1,0,0,127,2662,...,113,34,11,3,3790,351,345,NaN,NaN,3392
2,towerjo01,2009,27,5,10,15,0,0,0,337,...,3,1,7,0,494,8,76,3,5,456
3,hillesh01,1993,18,3,6,11,0,0,0,182,...,1,1,4,0,288,4,48,NaN,NaN,251
4,contrjo01,2013,136,13,17,23,0,0,9,663,...,6,8,12,0,966,33,129,15,4,849


In [3]:
rest_data["Exp ER | ERA = 4"] = (rest_data["ipouts"] / 3) * 4 / 9


In [4]:
rest_data["ER_Saved"] = rest_data["Exp ER | ERA = 4"] - rest_data["er"]


In [5]:
yvals = rest_data["ER_Saved"]
yvals.head()

0     6.111111
1    94.370370
2   -16.074074
3   -20.037037
4   -13.777778
Name: ER_Saved, dtype: float64

In [6]:
pitch["start_year_agg"] = np.floor((pitch["start_year"] - 1970)/5)
del pitch["start_year"]

In [7]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(pitch["start_year_agg"])
binarized = lb.transform(pitch["start_year_agg"])
bin_df = pd.DataFrame(binarized)
bin_df = bin_df.rename(columns={0:"1970-74", 1: "1975-79", 2: "1980-84", 3: "1985-89", 4: "1990-94", 5: "1995-99", 6: "2000-04", 7: "2005-09"})
del pitch["start_year_agg"]
pitch= pd.merge(pitch, bin_df, how='left', left_index=True, right_index=True)

In [8]:
pitch.head()

,player_id,g,w,l,gs,cg,sho,sv,ipouts,h,...,sf,tbf,1970-74,1975-79,1980-84,1985-89,1990-94,1995-99,2000-04,2005-09
0,gregelu01,435,22,27,0,0,0,19,1258,330,...,5,1540,0,0,0,0,0,0,0,1
1,mintogr01,141,10,15,6,0,0,23,730,227,...,NaN,919,0,1,0,0,0,0,0,0
2,towerjo01,112,40,45,97,4,2,1,1857,752,...,22,2504,0,0,0,0,0,0,1,0
3,hillesh01,163,21,32,51,1,1,10,1364,443,...,NaN,1763,0,0,0,1,0,0,0,0
4,contrjo01,163,65,50,152,5,3,0,2856,951,...,29,3677,0,0,0,0,0,0,1,0


In [9]:
features = list(pitch.columns)

In [10]:
features.remove('player_id')
features.remove('sf')
features.remove('sh')
features.remove('sho')
features.remove('cg')
features

['g',
 'w',
 'l',
 'gs',
 'sv',
 'ipouts',
 'h',
 'er',
 'hr',
 'bb',
 'so',
 'ibb',
 'wp',
 'hbp',
 'bk',
 'bfp',
 'gf',
 'r',
 'tbf',
 '1970-74',
 '1975-79',
 '1980-84',
 '1985-89',
 '1990-94',
 '1995-99',
 '2000-04',
 '2005-09']

In [11]:
pitch = pitch[features]
pitch.head()

,g,w,l,gs,sv,ipouts,h,er,hr,bb,...,r,tbf,1970-74,1975-79,1980-84,1985-89,1990-94,1995-99,2000-04,2005-09
0,435,22,27,0,19,1258,330,128,29,122,...,145,1540,0,0,0,0,0,0,0,1
1,141,10,15,6,23,730,227,89,4,96,...,107,919,0,1,0,0,0,0,0,0
2,112,40,45,97,1,1857,752,336,104,100,...,365,2504,0,0,0,0,0,0,1,0
3,163,21,32,51,10,1364,443,217,46,205,...,236,1763,0,0,0,1,0,0,0,0
4,163,65,50,152,0,2856,951,483,111,341,...,531,3677,0,0,0,0,0,0,1,0


In [12]:
X = pitch.as_matrix()
Y = yvals.as_matrix()
X[0:5]

array([[  4.35000000e+02,   2.20000000e+01,   2.70000000e+01,
          0.00000000e+00,   1.90000000e+01,   1.25800000e+03,
          3.30000000e+02,   1.28000000e+02,   2.90000000e+01,
          1.22000000e+02,   4.11000000e+02,   2.20000000e+01,
          1.60000000e+01,   1.40000000e+01,   0.00000000e+00,
          1.70200000e+03,   7.60000000e+01,   1.45000000e+02,
          1.54000000e+03,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.41000000e+02,   1.00000000e+01,   1.50000000e+01,
          6.00000000e+00,   2.30000000e+01,   7.30000000e+02,
          2.27000000e+02,   8.90000000e+01,   4.00000000e+00,
          9.60000000e+01,   9.90000000e+01,   1.80000000e+01,
          2.20000000e+01,   5.00000000e+00,   6.00000000e+00,
          1.02000000e+03,   6.40000000e+01,   1.07000000e+02,
          9.19000000e+02,   0.00000000e+00,   1.00000000e+00,
       

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25, random_state=54321)

Scaler = MinMaxScaler()
Scaler.fit(Xtrain)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [14]:
Xtrain = Scaler.transform(Xtrain)
Xtest = Scaler.transform(Xtest)
X = Scaler.transform(X)
X[0:5]

array([[ 0.97963801,  0.22      ,  0.31764706,  0.        ,  0.10106383,
         0.25732087,  0.2292406 ,  0.19808307,  0.15819209,  0.17647059,
         0.3081571 ,  0.31884058,  0.20253165,  0.17073171,  0.        ,
         0.25654047,  0.23676012,  0.2079646 ,  0.2490099 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ],
       [ 0.31447964,  0.1       ,  0.17647059,  0.02816901,  0.12234043,
         0.14766355,  0.15613911,  0.13578275,  0.01694915,  0.13823529,
         0.07250755,  0.26086957,  0.27848101,  0.06097561,  0.26086957,
         0.1515851 ,  0.19937695,  0.1519174 ,  0.14653465,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.24886878,  0.4       ,  0.52941176,  0.45539906,  0.00531915,
         0.38172378,  0.52874379,  0.53035144,  0.5819209 ,  0.14411765,
         0.2326284 ,  0.14492754,  0.06329114,  0.3414

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

DTR = DecisionTreeRegressor(random_state=1234)
mx_dpth = [1, 2, 3, 4, 5, 6, 7, 12]
mn_lf_splt = [1, 2, 3, 4, 5, 6, 7, 12]
param_dict_rf = {'max_depth': mx_dpth, 'min_samples_leaf' : mn_lf_splt}

In [16]:
clf_rf = GridSearchCV(DTR, param_dict_rf, return_train_score=True, scoring='r2')
clf_rf.fit(X, Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1234, splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 12], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [17]:
clf_rf.cv_results_


{'mean_fit_time': array([ 0.00279077,  0.00191991,  0.00186777,  0.00185402,  0.00160519,
         0.00128865,  0.00140818,  0.00128738,  0.00234373,  0.00231814,
         0.00230098,  0.0022831 ,  0.00241232,  0.00239404,  0.00226124,
         0.00224368,  0.00334764,  0.00334883,  0.00321595,  0.00348179,
         0.00328223,  0.00321476,  0.0033191 ,  0.00326443,  0.00420356,
         0.00426888,  0.00429384,  0.00416327,  0.00427906,  0.00430727,
         0.0041821 ,  0.00415532,  0.00529615,  0.00531419,  0.00519101,
         0.00517925,  0.00502459,  0.00518036,  0.00516335,  0.00504096,
         0.00641354,  0.00613467,  0.00599543,  0.00616026,  0.0060238 ,
         0.00604828,  0.00594211,  0.00581861,  0.0071222 ,  0.00729243,
         0.00706363,  0.00686534,  0.00676457,  0.00682529,  0.00674788,
         0.00667286,  0.01152237,  0.01077191,  0.01069729,  0.01030715,
         0.01002185,  0.00999244,  0.00974067,  0.00879288]),
 'mean_score_time': array([ 0.00064262,  0.00

In [54]:
clf_rf.best_params_
clf_rf.best_score_

0.025917037880519245

In [19]:
from sklearn.svm import SVR

svr = SVR(kernel = 'rbf')

In [20]:
Cval = [100, 100000, 1000000]
eps = [10, 100]
gamma = [0.001, 0.0001, 0.00001]
svm_dict = {'C': Cval, 'epsilon' : eps, 'gamma' : gamma}
clf_svm = GridSearchCV(svr, svm_dict, return_train_score=True, scoring='r2')
clf_svm.fit(X, Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [100, 100000, 1000000], 'epsilon': [10, 100], 'gamma': [0.001, 0.0001, 1e-05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [21]:
clf_svm.best_score_

0.098428249202965334

In [22]:
clf_svm.cv_results_

{'mean_fit_time': array([ 0.04734516,  0.04426424,  0.04466645,  0.00360934,  0.00359424,
         0.00396832,  0.10009694,  0.07840268,  0.05308461,  0.02976592,
         0.00474358,  0.00343919,  0.37817375,  0.12455837,  0.07175628,
         0.03922685,  0.02665655,  0.00442497]),
 'mean_score_time': array([ 0.01342726,  0.01343473,  0.01353081,  0.00120584,  0.00113781,
         0.00130018,  0.0144697 ,  0.01774375,  0.0143501 ,  0.00138402,
         0.00106923,  0.00114195,  0.01473284,  0.01497618,  0.01468905,
         0.00142924,  0.00130057,  0.00108568]),
 'mean_test_score': array([ 0.02608934,  0.00162285, -0.00345603, -0.0553562 , -0.05438133,
        -0.05433035,  0.09404239,  0.0785532 ,  0.05441726, -0.31919792,
        -0.14019829, -0.07328479,  0.09842825,  0.08855179,  0.07874048,
        -0.45880492, -0.31658692, -0.14159541]),
 'mean_train_score': array([ 0.037516  ,  0.00574796, -0.00067394, -0.05653504, -0.0562979 ,
        -0.05633628,  0.14916766,  0.11313941,  

In [23]:
clf_svm.best_params_

{'C': 1000000, 'epsilon': 10, 'gamma': 0.001}

In [44]:
from sklearn.neural_network import MLPRegressor
MLP = MLPRegressor(activation='relu', hidden_layer_sizes = (6), solver='lbfgs', alpha =1, random_state=54321)


param_grid = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
param_dict = {"alpha" : param_grid}
clf = GridSearchCV(MLP, param_dict, return_train_score=True, scoring='r2')

In [45]:
clf.fit(X, Y)

clf.cv_results_

{'mean_fit_time': array([ 0.07565896,  0.06467994,  0.06559801,  0.06548079,  0.06589127,
         0.06558291,  0.06676332,  0.06816848,  0.05808218,  0.00490991,
         0.00536895]),
 'mean_score_time': array([ 0.00036565,  0.000319  ,  0.00036303,  0.00029437,  0.00033482,
         0.0002927 ,  0.00033522,  0.00029755,  0.00030923,  0.00030788,
         0.0002679 ]),
 'mean_test_score': array([-0.11856707, -0.0797476 , -0.15925822, -0.06835729, -0.17708468,
        -0.07290506, -0.09929219,  0.04889043,  0.08582894, -0.00206667,
        -0.00206646]),
 'mean_train_score': array([  3.21204981e-01,   3.38792556e-01,   3.37147283e-01,
          3.27864555e-01,   3.43256678e-01,   3.33740590e-01,
          3.35663189e-01,   2.71657025e-01,   1.33811231e-01,
         -3.41499791e-11,  -6.14678678e-12]),
 'param_alpha': masked_array(data = [1e-05 0.0001 0.001 0.01 0.1 1 10 100 1000 10000 100000],
              mask = [False False False False False False False False False False False],
  

In [46]:
clf.best_params_

{'alpha': 1000}

In [47]:
from sklearn.linear_model import Ridge

ridge = Ridge(random_state=1234, solver = 'lsqr')
rdge_param = {'alpha' : [0.01, 0.1, 1, 10, 100, 1000, 10000, 100000, 1000000]}

clf_rdge = GridSearchCV(ridge, rdge_param, return_train_score=True, scoring='r2')

In [49]:
clf_rdge.fit(X,Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=1234, solver='lsqr', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.01, 0.1, 1, 10, 100, 1000, 10000, 100000, 1000000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [55]:
clf_rdge.best_score_

0.090428940654199735

In [56]:
clf_rdge.best_params_

{'alpha': 1}

In [57]:
clf_rdge.cv_results_

{'mean_fit_time': array([ 0.00678166,  0.00386802,  0.00599885,  0.00353845,  0.00177169,
         0.00163992,  0.00097815,  0.0016493 ,  0.00126902]),
 'mean_score_time': array([ 0.00038973,  0.00228715,  0.0003264 ,  0.00031217,  0.00027299,
         0.00038266,  0.00024652,  0.00027363,  0.00076882]),
 'mean_test_score': array([ 0.08508029,  0.08583344,  0.09042894,  0.08896563,  0.05928172,
         0.01261899, -0.00038525, -0.00189515, -0.00204951]),
 'mean_train_score': array([  1.77719206e-01,   1.75488072e-01,   1.66819265e-01,
          1.35818560e-01,   7.71915364e-02,   1.80194979e-02,
          2.10561634e-03,   2.14377114e-04,   2.14765475e-05]),
 'param_alpha': masked_array(data = [0.01 0.1 1 10 100 1000 10000 100000 1000000],
              mask = [False False False False False False False False False],
        fill_value = ?),
 'params': [{'alpha': 0.01},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 10},
  {'alpha': 100},
  {'alpha': 1000},
  {'alpha': 10000},
  {'alpha